In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# --------------------- Constants ---------------------
IMAGE_DIR = "cgr_images"
IMAGE_SIZE = (128, 128)
PCA_COMPONENTS = 150
EPOCHS = 50
BATCH_SIZE = 32
NUM_CLASSES = 2

os.makedirs("results", exist_ok=True)

# --------------------- Load Images & Labels ---------------------
X, y = [], []

for class_name in os.listdir(IMAGE_DIR):
    class_dir = os.path.join(IMAGE_DIR, class_name)
    if not os.path.isdir(class_dir) or class_name.startswith('.'):
        continue

    label = 0 if class_name.lower() == "nonhuman" else 1

    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')) or img_file.startswith('.'):
            continue
        try:
            img = Image.open(img_path).resize(IMAGE_SIZE).convert("RGB")
            img_array = np.array(img).flatten()
            X.append(img_array)
            y.append(label)
        except Exception as e:
            print(f"⚠️ Skipping {img_path}: {e}")

X = np.array(X)
y = np.array(y)
print("✅ Loaded data:", X.shape, y.shape)

# --------------------- Apply PCA ---------------------
print("🔍 Applying PCA...")
pca = PCA(n_components=PCA_COMPONENTS)
X_pca = pca.fit_transform(X)
X_pca = X_pca.reshape(-1, PCA_COMPONENTS, 1)
print("✅ PCA completed:", X_pca.shape)

# --------------------- Train/Test Split ---------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_pca, y, test_size=0.2, stratify=y, random_state=42
)

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=NUM_CLASSES)
y_val_cat = to_categorical(y_val, num_classes=NUM_CLASSES)

# --------------------- Compute Class Weights ---------------------
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("📊 Class Weights:", class_weights)

# --------------------- Build CNN Model ---------------------
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(PCA_COMPONENTS, 1), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),

    Conv1D(128, 3, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --------------------- Callbacks ---------------------
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)

# --------------------- Train ---------------------
history = model.fit(
    X_train, y_train_cat,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val_cat),
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr]
)

# Save model
model.save("results/pca_cnn_model.keras")

# --------------------- Plot Accuracy ---------------------
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid()
plt.savefig("results/pca_cnn_accuracy.png")
plt.close()

# --------------------- Evaluation ---------------------
y_pred = np.argmax(model.predict(X_val), axis=1)
labels = ["NonHuman", "Human"]

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig("results/pca_cnn_confusion_matrix.png")
plt.close()

# Classification Report
report = classification_report(y_val, y_pred, target_names=labels, digits=4)
print("📋 Final Classification Report:\n")
print(report)
with open("results/pca_cnn_classification_report.txt", "w") as f:
    f.write(report)

# Final Accuracy
final_acc = np.sum(y_pred == y_val) / len(y_val)
print(f"✅ Final Validation Accuracy: {final_acc * 100:.2f}%")


✅ Loaded data: (1000, 49152) (1000,)
🔍 Applying PCA...
✅ PCA completed: (1000, 150, 1)
📊 Class Weights: {0: np.float64(1.0), 1: np.float64(1.0)}


C:\Users\babua\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 148, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 148, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 74, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 74, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 72, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 72, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 36, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 615,938 (2.35 MB)

 Trainable params: 615,554 (2.35 MB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.8252 - loss: 0.7139 - val_accuracy: 0.9950 - val_loss: 0.1946 - learning_rate: 0.0010
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9958 - loss: 0.1198 - val_accuracy: 0.9900 - val_loss: 0.1441 - learning_rate: 0.0010
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9924 - loss: 0.1028 - val_accuracy: 0.9850 - val_loss: 0.1487 - learning_rate: 0.0010
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9924 - loss: 0.1146 - val_accuracy: 0.9800 - val_loss: 0.1703 - learning_rate: 0.0010
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9956 - loss: 0.0965 - val_accuracy: 0.9800 - val_loss: 0.1710 - learning_rate: 0.0010
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9988 - loss: 0.0871 - val_accuracy: 0.9800 - val_loss: 0.1454 - learning_rate: 3.0000e-04
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9958 - loss: 0.0959 - val